<a href="https://colab.research.google.com/github/ChangQingxgg/Document-Answering-Robot/blob/main/%E5%9B%BD%E5%86%85%E5%A4%A7%E6%A8%A1%E5%9E%8B%E5%BB%BA%E7%AB%8B%E6%95%B0%E6%8D%AE%E5%BA%93%E6%9C%80%E7%BB%88%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install datasets
!pip install openai
!pip install zhipuai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
import pandas as pd
from datasets import load_dataset
from openai import OpenAI
from zhipuai import ZhipuAI
import os
from google.colab import drive
drive.mount('/content/drive')

# API 密钥（请替换为你的实际 API 密钥）
DEEPSEEK_API_KEY = "sk-5bb4dcdbbbcb4d00a7a86c394b48f8cd"
ZHIPUAI_API_KEY = "027ca123cc5f4ced99d7a7d257d68531.9OT5hZQLmCOSKpjG"
TONGYI_API_KEY = "sk-7e39c80d2a2b4b049b90f534ef236062"

# 初始化客户端
deepseek_client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")
zhipuai_client = ZhipuAI(api_key=ZHIPUAI_API_KEY)
tongyi_client = OpenAI(api_key=TONGYI_API_KEY, base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

# 系统提示词，适用于所有模型
system_prompt = """You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response."""

# DeepSeek 模型生成回答
def generate_deepseek_response(question):
    try:
        response = deepseek_client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Please answer the following medical question: {question}"}
            ],
            max_tokens=2048,
            temperature=0.7,
            stream=False
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error generating DeepSeek response: {e}")
        return "Error: Unable to generate response."

# 智谱清言模型生成回答
def generate_zhipuai_response(question):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Please answer the following medical question: {question}"}
    ]
    try:
        response = zhipuai_client.chat.completions.create(
            model="glm-4-plus",
            messages=messages,
            max_tokens=2048,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error generating ZhipuAI response: {e}")
        return "Error: Unable to generate response."

# 通义千问模型生成回答
def generate_tongyi_response(question):
    try:
        completion = tongyi_client.chat.completions.create(
            model="qwen-plus",
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': f"Please answer the following medical question: {question}"}
            ],
            max_tokens=2048,
            temperature=0.7
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Error generating Tongyi response: {e}")
        return "Error: Unable to generate response."

# 批量处理函数，生成三个模型的回答
def add_generated_responses(batch):
    questions = batch["Question"]
    deepseek_responses = [generate_deepseek_response(q) for q in questions]
    zhipuai_responses = [generate_zhipuai_response(q) for q in questions]
    tongyi_responses = [generate_tongyi_response(q) for q in questions]
    return {
        "deepseek_response": deepseek_responses,
        "zhipuai_response": zhipuai_responses,
        "tongyi_response": tongyi_responses
    }

# 加载医疗数据集，使用流式加载
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split="train", trust_remote_code=True, streaming=True)

# 应用映射函数，添加三个模型的生成回答
dataset = dataset.map(add_generated_responses, batched=True, batch_size=8)

# 取前 3 个样本作为演示
dataset_subset = dataset.take(3)

# 收集数据到列表中
data = []
for example in dataset_subset:
    data.append({
        "question": example["Question"],
        "context": example["Response"],
        "deepseek_response": example["deepseek_response"],
        "zhipuai_response": example["zhipuai_response"],
        "tongyi_response": example["tongyi_response"]
    })

# 创建 Pandas DataFrame
df = pd.DataFrame(data)

# 保存 DataFrame 到 CSV 文件
df.to_csv("/content/drive/MyDrive/medical_responses_qwen.csv", index=False)
print("数据库已保存至 'medical_responses.csv'")

Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

数据库已保存至 'medical_responses.csv'
